In [ ]:
import math
import timeit
import random
from pprint import pprint
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

# Data Generator

## X generator

In [ ]:
def X_generator(n = 10000, p = 20):
  
  df = pd.DataFrame(np.random.randint(0, 1, size=(n, p)))

  for col_index in range(p):
    if col_index % 2 != 0:
      df[col_index] = np.random.binomial(1, 0.5, n)
    else:
      df[col_index] = np.random.normal(0, 1, n)

  return df

## W generator

In [ ]:
def W_generator(X):

  n, p = X.shape

  beta_0 = -3.0
  eta = pd.Series(np.zeros(n) + beta_0)

  for col_index in range(p):
    eta += X[col_index] * 1.0 / np.sqrt(col_index + 1)
  ZISC_propensity_score = np.exp(eta) / (1 + np.exp(eta))
  
  W = pd.Series(np.zeros(n)) * 1.0
  for i in range(n):
    W[i] = np.random.binomial(1, ZISC_propensity_score[i], 1)
  
  return W

## DGP: Data Generation Process

In [ ]:
def f_1(X):
  return (X[0] > 1).astype(float)

def f_2(X):
  return X[0]

def f_3(X):
  return X[0] + X[2] + X[4] + X[6] + X[7] + X[8] - 2.0

def f_4(X):
  return 1.0 * (X[0] > 1).astype(float) * (X[2] > 0).astype(float) + 2.0 * (X[4] > 1).astype(float) * (X[6] > 0).astype(float) + 3.0 * X[7] * X[8]

def f_5(X):
  return 1.0 * (~(X[1] != X[3])).astype(float) + 3.0 * (~(X[3] != X[5])).astype(float) + 5.0 * (~(X[5] != X[1])).astype(float)

def f_6(X):
  return X[0] **2 + X[1] + X[2] ** 2 + X[3] + X[4] ** 2 +  X[5] +  X[6] ** 2 + X[7] + X[8] ** 2

def f_7(X):
  return 0.5 * f_3(X) + 0.5 * f_5(X)

def f_8(X):
  return np.sin(math.pi * X[0] * X[1]) + 2 * X[2] * X[3] + 0.5 * X[4] * X[5]

DGP = {
    "f_1": f_1,
    "f_2": f_2,
    "f_3": f_3,
    "f_4": f_4,
    "f_5": f_5,
    "f_6": f_6,
    "f_7": f_7,
    "f_8": f_8
}

Tree_Optimal_Depth  = {
    "f_1": 8,
    "f_2": 8,
    "f_3": 6,
    "f_4": 4,
    "f_5": 7,
    "f_6": 6,
    "f_7": 6,
    "f_8": 6
}


## Training Data Generator

In [ ]:
def training_data(f = "f_1", n = 10000, p = 20):

  X = X_generator(n, p)
  W = W_generator(X)
  f_X = DGP[f](X)
  
  # Standardlized f_x
  f_X_standardized = (f_X - f_X.mean())/(f_X.std())
  # true z, which is the quantity we would like to estimate
  z = np.exp(f_X_standardized + 0.5)
  
  # generate observation y
  y = pd.Series(np.zeros(n)) * 1.0
  for i in range(n):
    y[i] = np.exp(np.random.normal(f_X_standardized[i], 1)) * W[i]

  # generate y_tilde = y / prepensity_score
  # prepensity_score estiamtion (propensity_score_hat) based on Logistic Regression
  logisticRegr = LogisticRegression()
  logisticRegr.fit(X, W)
  propensity_score_hat = logisticRegr.predict_proba(X)[:,1]
  y_tilde = y / propensity_score_hat
  
  return X.to_numpy(), z.to_numpy(), y.to_numpy(), y_tilde.to_numpy()

# SSL: Self-Training Regression

## Self-Training Wrapper

In [ ]:
def semi_self_training_LR(X_train, y_train, X_test):
    
  Regressor_with_labeled_data_only = LinearRegression()
  Regressor_with_labeled_data_only.fit(X_train[y_train>0], y_train[y_train>0])
  y_train_unlabel_pred = Regressor_with_labeled_data_only.predict(X_train[y_train==0])

  X_train_new = np.concatenate((X_train[y_train>0], X_train[y_train==0]), axis=0)
  y_train_new = np.concatenate((y_train[y_train>0], y_train_unlabel_pred), axis=0)

  Regressor_with_pred_labeled_data = LinearRegression()
  Regressor_with_pred_labeled_data.fit(X_train_new, y_train_new)
  
  return Regressor_with_pred_labeled_data.predict(X_test)

def semi_self_training_Tree(X_train, y_train, X_test, max_depth):
    
  Regressor_with_labeled_data_only = DecisionTreeRegressor(max_depth=max_depth)
  Regressor_with_labeled_data_only.fit(X_train[y_train>0], y_train[y_train>0])
  y_train_unlabel_pred = Regressor_with_labeled_data_only.predict(X_train[y_train==0])

  X_train_new = np.concatenate((X_train[y_train>0], X_train[y_train==0]), axis=0)
  y_train_new = np.concatenate((y_train[y_train>0], y_train_unlabel_pred), axis=0)

  Regressor_with_pred_labeled_data = DecisionTreeRegressor(max_depth=max_depth)
  Regressor_with_pred_labeled_data.fit(X_train_new, y_train_new)
  
  return Regressor_with_pred_labeled_data.predict(X_test)

def semi_self_training_Forest(X_train, y_train, X_test, max_depth):
    
  Regressor_with_labeled_data_only = RandomForestRegressor(n_estimators=100, max_features='sqrt', max_depth=max_depth)
  Regressor_with_labeled_data_only.fit(X_train[y_train>0], y_train[y_train>0])
  y_train_unlabel_pred = Regressor_with_labeled_data_only.predict(X_train[y_train==0])

  X_train_new = np.concatenate((X_train[y_train>0], X_train[y_train==0]), axis=0)
  y_train_new = np.concatenate((y_train[y_train>0], y_train_unlabel_pred), axis=0)

  Regressor_with_pred_labeled_data = RandomForestRegressor(n_estimators=100, max_features='sqrt', max_depth=max_depth)
  Regressor_with_pred_labeled_data.fit(X_train_new, y_train_new)
  
  return Regressor_with_pred_labeled_data.predict(X_test)

## Semi-Supervised Tree and Forest

### Tree Algo

In [ ]:
class Semi_Supervised_Tree(object):
    def __init__(self, weight = 0.5, splitter='best', max_depth = 10, min_num_of_pos = 2, min_num_of_zero = 1):
        self.depth = 0
        self.weight = weight
        self.max_depth = max_depth
        self.splitter = splitter
        self.min_num_of_pos = min_num_of_pos
        self.min_num_of_zero = min_num_of_zero
        self.trees = {}

    def fit(self, X, y, par_node = {}, depth = 0):
        if (depth == 0):
          self.X_variance = np.var(X, axis=0)
          self.y_plus_variance = np.var(y[y>0])

        n_0, n_plus, y_plus_mean, y_plus_var = self.basic_summary_statistics(y)
        col_index, cutoff, cost = self.find_best_split_of_all(X, y)
        
        if par_node is None:  # base case 1: tree stops at previous level
            return None
        elif (n_0 <= self.min_num_of_pos or n_plus <= self.min_num_of_zero): # base case 2: 100 zero or 50 positive instances in one node    
            return {'val_CME': y_plus_mean,  
                    'n_0': n_0,
                    'n_plus': n_plus,
                    'n': n_0 + n_plus,
                    'depth': depth}
        elif depth >= self.max_depth: # base case 3: max depth reached 
            return {'val_CME': y_plus_mean, 
                    'n_0': n_0,
                    'n_plus': n_plus,
                    'n': n_0 + n_plus,
                    'depth': depth}
        elif (cutoff == None): #base case 4: cannot split based one our rules
            return {'val_CME': y_plus_mean, 
                    'n_0': n_0,
                    'n_plus': n_plus,
                    'n': n_0 + n_plus,
                    'depth': depth}
        else: 
            y_left = y[X[:, col_index] < cutoff]
            y_right = y[X[:, col_index] >= cutoff]
            par_node = {'col_index': col_index,
                        'cutoff':cutoff,
                        'val_CME': y_plus_mean,
                        'n_0': n_0,
                        'n_plus': n_plus,
                        'n': n_0 + n_plus,
                        'depth': depth}
            par_node['left'] = self.fit(X[X[:, col_index] < cutoff], y_left, {}, depth+1)
            par_node['right'] = self.fit(X[X[:, col_index] >= cutoff], y_right, {}, depth+1)
            self.depth += 1 
            self.trees = par_node
            return par_node

    def basic_summary_statistics(self, y):
        """
        Return n_0, n_+, y_+_mean, y_+_var
        """
        y_0 = y[y == 0]
        y_plus = y[y > 0]
        n_0 = len(y_0)
        n_plus = len(y_plus)
        y_plus_mean = np.mean(y_plus)
        y_plus_var = np.var(y_plus)
        
        return n_0, n_plus, y_plus_mean, y_plus_var
    
    def find_best_split_of_all(self, X, y):
        col_index = None
        min_cost = float("inf")
        cutoff = None

        num_features = len(X.T)
        if self.splitter == 'sqrt':
          num_features_for_split = int(np.sqrt(num_features))
        else:
          num_features_for_split = int(num_features)

        random_indices = np.random.choice(num_features, size=num_features_for_split, replace=False)

        for i, col in enumerate(X.T):
          if (i in random_indices):
            #print("col index: " + str(i))
            cur_cost, cur_cutoff = self.find_best_split(col, X, y)
            if cur_cost <= min_cost:
                min_cost = cur_cost
                col_index = i
                cutoff = cur_cutoff
        #print("local best col: " + str(col))
        #print("local best cutoff: " + str(cutoff))
        #print("local best min_cost: " + str(min_cost))
        return col_index, cutoff, min_cost

    def find_best_split(self, col, X, y):
        min_cost = float("inf")
        cutoff = None

        n_0_L, n_plus_L, y_plus_sum_L, y_plus_squared_sum_L = 0, 0, 0, 0
        n_0_R = sum(y == 0)
        n_plus_R = sum(y != 0)
        y_plus_sum_R = sum(y)
        y_plus_squared_sum_R = sum(y ** 2)

        X_sum_L = np.zeros(X.shape[1])
        X_squared_sum_L = np.zeros(X.shape[1])
        X_sum_R = X.sum(axis=0)
        X_squared_sum_R = np.square(X).sum(axis=0)

        sorted_col_index = np.argsort(col)
        sorted_X, sorted_y, sorted_col = X[sorted_col_index], y[sorted_col_index], col[sorted_col_index]

        for i in range(len(col)):
            if i == 0:
              continue
            else:
              to_be_moved_X = sorted_X[i - 1]
              X_sum_L += to_be_moved_X
              X_squared_sum_L += np.square(to_be_moved_X)
              X_sum_R -= to_be_moved_X
              X_squared_sum_R -= np.square(to_be_moved_X)
              
              to_be_moved_y = sorted_y[i - 1]              
              if (to_be_moved_y == 0):
                n_0_L += 1
                n_0_R -= 1
              else:
                n_plus_L += 1
                n_plus_R -= 1 
                y_plus_sum_L += to_be_moved_y
                y_plus_squared_sum_L += to_be_moved_y ** 2
                y_plus_sum_R -= to_be_moved_y
                y_plus_squared_sum_R -= to_be_moved_y ** 2             
            if (n_0_L < 1 or n_plus_L < 2 or n_0_R < 1 or n_plus_R < 2):
              continue
            if sorted_col[i-1] == sorted_col[i]:
              continue
            current_cost = self.ssl_cost_of_one_node(X_sum_L, X_squared_sum_L, n_0_L, n_plus_L, y_plus_sum_L, y_plus_squared_sum_L) + self.ssl_cost_of_one_node(X_sum_R, X_squared_sum_R, n_0_R, n_plus_R, y_plus_sum_R, y_plus_squared_sum_R)
            #print("cut off value: " + str(sorted_col[i]))
            #print("current cost: " + str(current_cost))
            if current_cost <= min_cost:
                min_cost = current_cost
                cutoff = sorted_col[i]
        return min_cost, cutoff

    def ssl_cost_of_one_node(self, X_sum, X_squared_sum, n_0, n_plus, y_plus_sum, y_plus_squared_sum):
        n = n_0 + n_plus
        cost_y_num = y_plus_squared_sum / n_plus -  (y_plus_sum / n_plus)**2
        cost_y = cost_y_num / self.y_plus_variance

        cost_X = 0
        cost_X_num = X_squared_sum / n - np.square(X_sum / n)
        p = len(cost_X_num)
        for i in range(p):
          cost_X += cost_X_num[i] / self.X_variance[i]

        return n * (self.weight * cost_y + (1 - self.weight) / p * cost_X)

    def predict_CME(self, X):
        results = np.array([0.0]*len(X))
        for i, c in enumerate(X):
            results[i] = self._get_prediction_CME(c)
        return results
    
    def _get_prediction_CME(self, row):
        cur_layer = self.trees
        while cur_layer.get('cutoff'):
            if row[cur_layer['col_index']] < cur_layer['cutoff']:
                cur_layer = cur_layer['left']
            else:
                cur_layer = cur_layer['right']
        else:
            return cur_layer.get('val_CME')

### Forest Algo

In [ ]:
def SSRF(X_train, y_train, X_test, 
         weight = 0.5, splitter='sqrt', max_depth=10, min_num_of_pos=2, min_num_of_zero=1, num_trees=100):
  
  SSRF_predicted = np.zeros(X_test.shape[0])

  for tree_index in range(0, num_trees):
    Semi_Supervised_Tree_ = Semi_Supervised_Tree(weight=weight, splitter=splitter, max_depth=max_depth, min_num_of_pos=min_num_of_pos, min_num_of_zero=min_num_of_zero)
    Semi_Supervised_Tree_.fit(X_train, y_train)
    SSRF_predicted += Semi_Supervised_Tree_.predict_CME(X_test)

  return SSRF_predicted / num_trees

# Semi-Continuous Tree and Random Forest



## Tree Algo

In [ ]:
class Semi_Continuous_Tree(object):
    def __init__(self, splitting_criteria = "SSE", splitter='best', max_depth = 10, min_num_of_pos = 2, min_num_of_zero = 1):
        self.depth = 0
        self.splitting_criteria = splitting_criteria
        self.max_depth = max_depth
        self.splitter = splitter
        self.min_num_of_pos = min_num_of_pos
        self.min_num_of_zero = min_num_of_zero
        self.trees = {}

    def fit(self, X, y, y_tilde, y_for_split, par_node = {}, depth = 0):
        n_0, n_plus, y_plus_mean, y_plus_var = self.basic_summary_statistics(y)
        col, cutoff, cost = self.find_best_split_of_all(X, y_for_split)
        if par_node is None:  # base case 1: tree stops at previous level
            return None
        elif (n_0 <= self.min_num_of_pos or n_plus <= self.min_num_of_zero): # base case 2: 100 zero or 50 positive instances in one node    
            return {'val_CME': y_plus_mean, 
                    'val_TOE': np.mean(y_tilde), 
                    'n_0': n_0,
                    'n_plus': n_plus,
                    'n': n_0 + n_plus,
                    'depth': depth}
        elif depth >= self.max_depth: # base case 3: max depth reached 
            return {'val_CME': y_plus_mean, 
                    'val_TOE': np.mean(y_tilde), 
                    'n_0': n_0,
                    'n_plus': n_plus,
                    'n': n_0 + n_plus,
                    'depth': depth}
        elif (cutoff == None): #base case 4: cannot split based one our rules
            return {'val_CME': y_plus_mean, 
                    'val_TOE': np.mean(y_tilde), 
                    'n_0': n_0,
                    'n_plus': n_plus,
                    'n': n_0 + n_plus,
                    'depth': depth}
        else: 
            y_left = y[X[:, col] < cutoff]
            y_right = y[X[:, col] >= cutoff]
            y_tilde_left = y_tilde[X[:, col] < cutoff]
            y_tilde_right = y_tilde[X[:, col] >= cutoff]
            y_for_split_left = y_for_split[X[:, col] < cutoff]
            y_for_split_right = y_for_split[X[:, col] >= cutoff]
            par_node = {'col_index': col,
                        'cutoff':cutoff,
                        'val_CME': y_plus_mean,
                        'val_TOE': np.mean(y_tilde),
                        'n_0': n_0,
                        'n_plus': n_plus,
                        'n': n_0 + n_plus,
                        'depth': depth}
            par_node['left'] = self.fit(X[X[:, col] < cutoff], y_left, y_tilde_left, y_for_split_left, {}, depth+1)
            par_node['right'] = self.fit(X[X[:, col] >= cutoff], y_right, y_tilde_right, y_for_split_right, {}, depth+1)
            self.depth += 1 
            self.trees = par_node
            return par_node
    
    def find_best_split_of_all(self, X, y):
        col = None
        min_cost = float("inf")
        cutoff = None

        num_features = len(X.T)
        if self.splitter == 'sqrt':
          num_features_for_split = int(np.sqrt(num_features))
        else:
          num_features_for_split = int(num_features)

        random_indices = np.random.choice(num_features, size=num_features_for_split, replace=False)

        for i, c in enumerate(X.T):
          if (i in random_indices):
            #print("col index: " + str(i))
            cur_cost, cur_cutoff = self.find_best_split(c, y)
            if cur_cost <= min_cost:
                min_cost = cur_cost
                col = i
                cutoff = cur_cutoff
        #print("local best col: " + str(col))
        #print("local best cutoff: " + str(cutoff))
        #print("local best min_cost: " + str(min_cost))
        return col, cutoff, min_cost

    def find_best_split(self, col, y):
        min_cost = float("inf")
        cutoff = None

        n_0_L, n_plus_L, y_plus_sum_L, y_plus_squared_sum_L = 0, 0, 0, 0
        n_0_R = sum(y == 0)
        n_plus_R = sum(y != 0)
        y_plus_sum_R = sum(y)
        y_plus_squared_sum_R = sum(y ** 2)

        sorted_col_index = np.argsort(col)
        sorted_y, sorted_col = y[sorted_col_index], col[sorted_col_index]

        for i in range(len(col)):
            if i == 0:
              continue
            else:
              to_be_moved_y = sorted_y[i - 1]
              if (to_be_moved_y == 0):
                n_0_L += 1
                n_0_R -= 1
              else:
                n_plus_L += 1
                n_plus_R -= 1 
                y_plus_sum_L += to_be_moved_y
                y_plus_squared_sum_L += to_be_moved_y ** 2
                y_plus_sum_R -= to_be_moved_y
                y_plus_squared_sum_R -= to_be_moved_y ** 2             
            if (n_0_L < 1 or n_plus_L < 2 or n_0_R < 1 or n_plus_R < 2):
              continue
            if sorted_col[i-1] == sorted_col[i]:
              continue
            cost_fn = self.get_cost(self.splitting_criteria)
            current_cost = cost_fn(n_0_L, n_plus_L, y_plus_sum_L, y_plus_squared_sum_L, n_0_R, n_plus_R, y_plus_sum_R, y_plus_squared_sum_R)
            #print("cut off value: " + str(sorted_col[i]))
            #print("current cost: " + str(current_cost))
            if current_cost <= min_cost:
                min_cost = current_cost
                cutoff = sorted_col[i]
        return min_cost, cutoff

    def predict_TOE(self, X):
        results = np.array([0.0]*len(X))
        for i, c in enumerate(X):
            results[i] = self._get_prediction_TOE(c)
        return results
    
    def _get_prediction_TOE(self, row):
        cur_layer = self.trees
        while cur_layer.get('cutoff'):
            if row[cur_layer['col_index']] < cur_layer['cutoff']:
                cur_layer = cur_layer['left']
            else:
                cur_layer = cur_layer['right']
        else:
            return cur_layer.get('val_TOE')

    def predict_CME(self, X):
        results = np.array([0.0]*len(X))
        for i, c in enumerate(X):
            results[i] = self._get_prediction_CME(c)
        return results
    
    def _get_prediction_CME(self, row):
        cur_layer = self.trees
        while cur_layer.get('cutoff'):
            if row[cur_layer['col_index']] < cur_layer['cutoff']:
                cur_layer = cur_layer['left']
            else:
                cur_layer = cur_layer['right']
        else:
            return cur_layer.get('val_CME')

    #  Non-self Functions
    def basic_summary_statistics(self, y):
      """
      Return n_0, n_+, y_+_mean, y_+_var
      """
      y_0 = y[y == 0]
      y_plus = y[y > 0]
      n_0 = len(y_0)
      n_plus = len(y_plus)
      y_plus_mean = np.mean(y_plus)
      y_plus_var = np.var(y_plus)

      return n_0, n_plus, y_plus_mean, y_plus_var

    def SSE_of_one_node(self, n_0, n_plus, y_plus_sum, y_plus_squared_sum):
      """
      Return SSE (Sum of Squared Error) of a node
      """

      return y_plus_squared_sum - y_plus_sum ** 2 / (n_0 + n_plus)


    def get_SSE(self, n_0_L, n_plus_L, y_plus_sum_L, y_plus_squared_sum_L, n_0_R, n_plus_R, y_plus_sum_R, y_plus_squared_sum_R):
      """
      Return SSE of a split
      """

      return self.SSE_of_one_node(n_0_L, n_plus_L, y_plus_sum_L, y_plus_squared_sum_L) + self.SSE_of_one_node(n_0_R, n_plus_R, y_plus_sum_R, y_plus_squared_sum_R)


    def get_t_squared(self, n_0_L, n_plus_L, y_plus_sum_L, y_plus_squared_sum_L, n_0_R, n_plus_R, y_plus_sum_R, y_plus_squared_sum_R):
      """
      Return t^2 of a split
      """

      y_plus_mean_L = y_plus_sum_L / n_plus_L
      y_plus_mean_R = y_plus_sum_R / n_plus_R
      sigma_squared_pool = (y_plus_squared_sum_L - (y_plus_sum_L ** 2 / n_plus_L) + y_plus_squared_sum_R - (y_plus_sum_R ** 2 / n_plus_R)) / (n_0_L + n_plus_L + n_0_R + n_plus_R - 4)
      t_squared = -1.0 * (y_plus_mean_L - y_plus_mean_R) ** 2 / (sigma_squared_pool * (1 / n_0_L + 1 / n_plus_L + 1 / n_0_R + 1 / n_plus_R))
      
      return t_squared

    def get_causal_cost(self, n_0_L, n_plus_L, y_plus_sum_L, y_plus_squared_sum_L, n_0_R, n_plus_R, y_plus_sum_R, y_plus_squared_sum_R):
      """
      Return causal cost of a split
      """
      y_plus_mean_L = y_plus_sum_L / n_plus_L
      y_plus_mean_R = y_plus_sum_R / n_plus_R

      causal_cost = -1.0 * ((n_0_L + n_plus_L) * y_plus_mean_L**2 + (n_0_R + n_plus_R) * y_plus_mean_R**2)

      return causal_cost

    def get_chi_squared_2(self, n_0_L, n_plus_L, y_plus_sum_L, y_plus_squared_sum_L, n_0_R, n_plus_R, y_plus_sum_R, y_plus_squared_sum_R):
      """
      Return chi^2_2 of a split
      """
      
      y_plus_mean_L = y_plus_sum_L / n_plus_L
      y_plus_mean_R = y_plus_sum_R / n_plus_R

      sigma_squared_pool_plus = (y_plus_squared_sum_L - (y_plus_sum_L ** 2 / n_plus_L) + y_plus_squared_sum_R - (y_plus_sum_R ** 2 / n_plus_R)) / (n_plus_L + n_plus_R - 2)
      positive_part_statistic = (y_plus_mean_L - y_plus_mean_R)**2 / ( sigma_squared_pool_plus * (1 / n_plus_L + 1 / n_plus_R))
      
      p_hat_L = n_0_L / (n_0_L + n_plus_L)
      p_hat_R = n_0_R / (n_0_R + n_plus_R)
      p_hat = (n_0_L + n_0_R) / (n_0_L + n_plus_L + n_0_R + n_plus_R)
      zero_part_statistic = (p_hat_L - p_hat_R) ** 2 / (p_hat * (1 - p_hat) * (1 / (n_0_L + n_plus_L) + 1 / (n_0_R + n_plus_R)))
      
      return -1.0 * (positive_part_statistic + zero_part_statistic)

    def get_cost(self, splitting_criteria):
      if splitting_criteria == 'SSE':
        return self.get_SSE
      elif splitting_criteria == 't_squared':
        return self.get_t_squared
      elif splitting_criteria == 'causal':
        return self.get_causal_cost
      elif splitting_criteria == 'chi_squared_2':
        return self.get_chi_squared_2

## Random Forest Algo

In [ ]:
def SCRF(X_train, y_train, y_tilde_train, y_for_split, X_test, 
         estimation, splitting_criteria, splitter='sqrt', 
         max_depth=10, min_num_of_pos=2, min_num_of_zero=1, num_trees=100):
  
  SCRF_predicted = np.zeros(X_test.shape[0])

  for tree_index in range(0, num_trees):
    Semi_Continuous_Tree_ = Semi_Continuous_Tree(splitting_criteria, splitter, max_depth, min_num_of_pos, min_num_of_zero)
    Semi_Continuous_Tree_.fit(X_train, y_train, y_tilde_train, y_for_split)
    z_test_predicted = np.zeros(X_test.shape[0])
    if (estimation == 'CME'):
      SCRF_predicted += Semi_Continuous_Tree_.predict_CME(X_test)
    elif (estimation == 'TOE'):
      SCRF_predicted += Semi_Continuous_Tree_.predict_TOE(X_test)
  
  return SCRF_predicted / num_trees


# Simulation

## RMSE Function

In [ ]:
def RMSE(z_hat, z):
  return np.sqrt(sum((z_hat - z) ** 2) / len(z))

## RMSE Dict

In [ ]:
Synthetic_Data_RMSE_Dict = {
  "CMR_RMSE": 
  {"f_1": [],"f_2": [],"f_3": [],"f_4": [],"f_5": [],"f_6": [],"f_7": [],"f_8": []},
  "SSL_ST_RF_RMSE":
  {"f_1": [],"f_2": [],"f_3": [],"f_4": [],"f_5": [],"f_6": [],"f_7": [],"f_8": []},
  "SSL_SSRF_RMSE":
  {"f_1": [],"f_2": [],"f_3": [],"f_4": [],"f_5": [],"f_6": [],"f_7": [],"f_8": []},
  "y_SSE_CME_RMSE":
  {"f_1": [],"f_2": [],"f_3": [],"f_4": [],"f_5": [],"f_6": [],"f_7": [],"f_8": []},
  "y_SSE_TOE_RMSE":
  {"f_1": [],"f_2": [],"f_3": [],"f_4": [],"f_5": [],"f_6": [],"f_7": [],"f_8": []},
  "y_t_squared_CME_RMSE":
  {"f_1": [],"f_2": [],"f_3": [],"f_4": [],"f_5": [],"f_6": [],"f_7": [],"f_8": []},
  "y_t_squared_TOE_RMSE":
  {"f_1": [],"f_2": [],"f_3": [],"f_4": [],"f_5": [],"f_6": [],"f_7": [],"f_8": []},
  "y_causal_CME_RMSE":
  {"f_1": [],"f_2": [],"f_3": [],"f_4": [],"f_5": [],"f_6": [],"f_7": [],"f_8": []},
  "y_causal_TOE_RMSE":
  {"f_1": [],"f_2": [],"f_3": [],"f_4": [],"f_5": [],"f_6": [],"f_7": [],"f_8": []},
  "y_chi_squared_2_CME_RMSE":
  {"f_1": [],"f_2": [],"f_3": [],"f_4": [],"f_5": [],"f_6": [],"f_7": [],"f_8": []},
  "y_chi_squared_2_TOE_RMSE":
  {"f_1": [],"f_2": [],"f_3": [],"f_4": [],"f_5": [],"f_6": [],"f_7": [],"f_8": []},
  "y_tilde_SSE_CME_RMSE":
  {"f_1": [],"f_2": [],"f_3": [],"f_4": [],"f_5": [],"f_6": [],"f_7": [],"f_8": []},
  "y_tilde_SSE_TOE_RMSE":
  {"f_1": [],"f_2": [],"f_3": [],"f_4": [],"f_5": [],"f_6": [],"f_7": [],"f_8": []},
  "y_tilde_t_squared_CME_RMSE":
  {"f_1": [],"f_2": [],"f_3": [],"f_4": [],"f_5": [],"f_6": [],"f_7": [],"f_8": []},
  "y_tilde_t_squared_TOE_RMSE":
  {"f_1": [],"f_2": [],"f_3": [],"f_4": [],"f_5": [],"f_6": [],"f_7": [],"f_8": []},
  "y_tilde_causal_CME_RMSE":
  {"f_1": [],"f_2": [],"f_3": [],"f_4": [],"f_5": [],"f_6": [],"f_7": [],"f_8": []},
  "y_tilde_causal_TOE_RMSE":
  {"f_1": [],"f_2": [],"f_3": [],"f_4": [],"f_5": [],"f_6": [],"f_7": [],"f_8": []},
  "y_tilde_chi_squared_2_CME_RMSE":
  {"f_1": [],"f_2": [],"f_3": [],"f_4": [],"f_5": [],"f_6": [],"f_7": [],"f_8": []},
  "y_tilde_chi_squared_2_TOE_RMSE":
  {"f_1": [],"f_2": [],"f_3": [],"f_4": [],"f_5": [],"f_6": [],"f_7": [],"f_8": []},
}

## Simulation Loop

In [ ]:
start = timeit.default_timer()
trail = 100
for t in range(trail):
  print('Trail: ' + str(t))
  for f in Synthetic_Data_RMSE_Dict['CMR_RMSE'].keys():
    
    print(f)
    
    np.random.seed(t)
    X_train, z_train, y_train, y_tilde_train = training_data(f)
    X_test, z_test, y_test, y_tilde_test = training_data(f)

    print("CMR")
    CMR_Forest = RandomForestRegressor(n_estimators=100, max_features='sqrt', max_depth=Tree_Optimal_Depth[f])
    CMR_Forest.fit(X_train[y_train>0], y_train[y_train>0])
    z_test_hat_CMR_Forest = CMR_Forest.predict(X_test)
    Synthetic_Data_RMSE_Dict["CMR_RMSE"][f].append(RMSE(z_test_hat_CMR_Forest, z_test))

    print("SSL ST RF")
    z_test_hat_SSL_ST_RF = semi_self_training_Forest(X_train, y_train, X_test, Tree_Optimal_Depth[f])
    Synthetic_Data_RMSE_Dict["SSL_ST_RF_RMSE"][f].append(RMSE(z_test_hat_SSL_ST_RF, z_test))

    print("SSL SSRF")
    z_test_hat_SSL_SSRF = SSRF(X_train, y_train, X_test, 0.5, 'sqrt',  Tree_Optimal_Depth[f])
    Synthetic_Data_RMSE_Dict["SSL_SSRF_RMSE"][f].append(RMSE(z_test_hat_SSL_SSRF, z_test))
    
    print("y_SSE_CME_RMSE")
    z_test_hat_y_SSE_CME= SCRF(X_train, y_train, y_tilde_train, y_train, X_test, 'CME', 'SSE', 'sqrt', Tree_Optimal_Depth[f])
    Synthetic_Data_RMSE_Dict['y_SSE_CME_RMSE'][f].append(RMSE(z_test_hat_y_SSE_CME, z_test))


    print("y_SSE_TOE_RMSE")
    z_test_hat_y_SSE_TOE= SCRF(X_train, y_train, y_tilde_train, y_train, X_test, 'TOE', 'SSE', 'sqrt', Tree_Optimal_Depth[f])
    Synthetic_Data_RMSE_Dict['y_SSE_TOE_RMSE'][f].append(RMSE(z_test_hat_y_SSE_TOE, z_test))


    print("y_t_squared_CME_RMSE")
    z_test_hat_y_t_squared_CME= SCRF(X_train, y_train, y_tilde_train, y_train, X_test, 'CME', 't_squared', 'sqrt', Tree_Optimal_Depth[f])
    Synthetic_Data_RMSE_Dict['y_t_squared_CME_RMSE'][f].append(RMSE(z_test_hat_y_t_squared_CME, z_test))


    print("y_t_squared_TOE_RMSE")
    z_test_hat_y_t_squared_TOE= SCRF(X_train, y_train, y_tilde_train, y_train, X_test, 'TOE', 't_squared', 'sqrt', Tree_Optimal_Depth[f])
    Synthetic_Data_RMSE_Dict['y_t_squared_TOE_RMSE'][f].append(RMSE(z_test_hat_y_t_squared_TOE, z_test))


    print("y_causal_CME_RMSE")
    z_test_hat_y_causal_CME= SCRF(X_train, y_train, y_tilde_train, y_train, X_test, 'CME', 'causal', 'sqrt', Tree_Optimal_Depth[f])
    Synthetic_Data_RMSE_Dict['y_causal_CME_RMSE'][f].append(RMSE(z_test_hat_y_causal_CME, z_test))


    print("y_causal_TOE_RMSE")
    z_test_hat_y_causal_TOE= SCRF(X_train, y_train, y_tilde_train, y_train, X_test, 'TOE', 'causal', 'sqrt', Tree_Optimal_Depth[f])
    Synthetic_Data_RMSE_Dict['y_causal_TOE_RMSE'][f].append(RMSE(z_test_hat_y_causal_TOE, z_test))


    print("y_chi_squared_2_CME_RMSE")
    z_test_hat_y_chi_squared_2_CME= SCRF(X_train, y_train, y_tilde_train, y_train, X_test, 'CME', 'chi_squared_2', 'sqrt', Tree_Optimal_Depth[f])
    Synthetic_Data_RMSE_Dict['y_chi_squared_2_CME_RMSE'][f].append(RMSE(z_test_hat_y_chi_squared_2_CME, z_test))


    print("y_chi_squared_2_TOE_RMSE")
    z_test_hat_y_chi_squared_2_TOE= SCRF(X_train, y_train, y_tilde_train, y_train, X_test, 'TOE', 'chi_squared_2', 'sqrt', Tree_Optimal_Depth[f])
    Synthetic_Data_RMSE_Dict['y_chi_squared_2_TOE_RMSE'][f].append(RMSE(z_test_hat_y_chi_squared_2_TOE, z_test))


    print("y_tilde_SSE_CME_RMSE")
    z_test_hat_y_tilde_SSE_CME= SCRF(X_train, y_train, y_tilde_train, y_tilde_train, X_test, 'CME', 'SSE', 'sqrt', Tree_Optimal_Depth[f])
    Synthetic_Data_RMSE_Dict['y_tilde_SSE_CME_RMSE'][f].append(RMSE(z_test_hat_y_tilde_SSE_CME, z_test))


    print("y_tilde_SSE_TOE_RMSE")
    z_test_hat_y_tilde_SSE_TOE= SCRF(X_train, y_train, y_tilde_train, y_tilde_train, X_test, 'TOE', 'SSE', 'sqrt', Tree_Optimal_Depth[f])
    Synthetic_Data_RMSE_Dict['y_tilde_SSE_TOE_RMSE'][f].append(RMSE(z_test_hat_y_tilde_SSE_TOE, z_test))


    print("y_tilde_t_squared_CME_RMSE")
    z_test_hat_y_tilde_t_squared_CME= SCRF(X_train, y_train, y_tilde_train, y_tilde_train, X_test, 'CME', 't_squared', 'sqrt', Tree_Optimal_Depth[f])
    Synthetic_Data_RMSE_Dict['y_tilde_t_squared_CME_RMSE'][f].append(RMSE(z_test_hat_y_tilde_t_squared_CME, z_test))


    print("y_tilde_t_squared_TOE_RMSE")
    z_test_hat_y_tilde_t_squared_TOE= SCRF(X_train, y_train, y_tilde_train, y_tilde_train, X_test, 'TOE', 't_squared', 'sqrt', Tree_Optimal_Depth[f])
    Synthetic_Data_RMSE_Dict['y_tilde_t_squared_TOE_RMSE'][f].append(RMSE(z_test_hat_y_tilde_t_squared_TOE, z_test))


    print("y_tilde_causal_CME_RMSE")
    z_test_hat_y_tilde_causal_CME= SCRF(X_train, y_train, y_tilde_train, y_tilde_train, X_test, 'CME', 'causal', 'sqrt', Tree_Optimal_Depth[f])
    Synthetic_Data_RMSE_Dict['y_tilde_causal_CME_RMSE'][f].append(RMSE(z_test_hat_y_tilde_causal_CME, z_test))


    print("y_tilde_causal_TOE_RMSE")
    z_test_hat_y_tilde_causal_TOE= SCRF(X_train, y_train, y_tilde_train, y_tilde_train, X_test, 'TOE', 'causal', 'sqrt', Tree_Optimal_Depth[f])
    Synthetic_Data_RMSE_Dict['y_tilde_causal_TOE_RMSE'][f].append(RMSE(z_test_hat_y_tilde_causal_TOE, z_test))


    print("y_tilde_chi_squared_2_CME_RMSE")
    z_test_hat_y_tilde_chi_squared_2_CME= SCRF(X_train, y_train, y_tilde_train, y_tilde_train, X_test, 'CME', 'chi_squared_2', 'sqrt', Tree_Optimal_Depth[f])
    Synthetic_Data_RMSE_Dict['y_tilde_chi_squared_2_CME_RMSE'][f].append(RMSE(z_test_hat_y_tilde_chi_squared_2_CME, z_test))


    print("y_tilde_chi_squared_2_TOE_RMSE")
    z_test_hat_y_tilde_chi_squared_2_TOE= SCRF(X_train, y_train, y_tilde_train, y_tilde_train, X_test, 'TOE', 'chi_squared_2', 'sqrt', Tree_Optimal_Depth[f])
    Synthetic_Data_RMSE_Dict['y_tilde_chi_squared_2_TOE_RMSE'][f].append(RMSE(z_test_hat_y_tilde_chi_squared_2_TOE, z_test))

    if (t % 5 == 0):
      pprint(Synthetic_Data_RMSE_Dict)

stop = timeit.default_timer()
print('Time: ', stop - start) 

## Print Results

In [ ]:
for model in Synthetic_Data_RMSE_Dict.keys():
  print(model)
  res = ""
  for f in Synthetic_Data_RMSE_Dict['CMR_RMSE'].keys():
      mean = np.mean(Synthetic_Data_RMSE_Dict[model][f])
      mean = np.round(mean, 2)
      std = np.std(Synthetic_Data_RMSE_Dict[model][f])
      std = np.round(std, 2)
      res += "& $ " + str(mean) + " \pm " + str(std) + " $"
  res += " \\\\"
  print(res)